<a href="https://colab.research.google.com/github/kstyle2198/NLP_TIPS/blob/main/RAG_with_Ko_PlatYi_6B(%EB%AA%A8%EB%91%90%EC%9D%98AI).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google Colab으로 오픈소스 LLM 구동하기

## 1단계 - LLM 양자화에 필요한 패키지 설치
- bitsandbytes: Bitsandbytes는 CUDA 사용자 정의 함수, 특히 8비트 최적화 프로그램, 행렬 곱셈(LLM.int8()) 및 양자화 함수에 대한 경량 래퍼
- PEFT(Parameter-Efficient Fine-Tuning): 모델의 모든 매개변수를 미세 조정하지 않고도 사전 훈련된 PLM(언어 모델)을 다양한 다운스트림 애플리케이션에 효율적으로 적용 가능
- accelerate: PyTorch 모델을 더 쉽게 여러 컴퓨터나 GPU에서 사용할 수 있게 해주는 도구


In [1]:
#양자화에 필요한 패키지 설치
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 8.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 1.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


## 2단계 - 트랜스포머에서 BitsandBytesConfig를 통해 양자화 매개변수 정의하기


* load_in_4bit=True: 모델을 4비트 정밀도로 변환하고 로드하도록 지정
* bnb_4bit_use_double_quant=True: 메모리 효율을 높이기 위해 중첩 양자화를 사용하여 추론 및 학습
* bnd_4bit_quant_type="nf4": 4비트 통합에는 2가지 양자화 유형인 FP4와 NF4가 제공됨. NF4 dtype은 Normal Float 4를 나타내며 QLoRA 백서에 소개되어 있습니다. 기본적으로 FP4 양자화 사용
* bnb_4bit_compute_dype=torch.bfloat16: 계산 중 사용할 dtype을 변경하는 데 사용되는 계산 dtype. 기본적으로 계산 dtype은 float32로 설정되어 있지만 계산 속도를 높이기 위해 bf16으로 설정 가능



In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

## 3단계 - 경량화 모델 로드하기

이제 모델 ID를 지정한 다음 이전에 정의한 양자화 구성으로 로드합니다.

In [3]:
model_id = "kyujinpy/Ko-PlatYi-6B"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")

tokenizer_config.json:   0%|          | 0.00/9.62k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.28M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/573 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/670 [00:00<?, ?B/s]

...

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.99G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/2.37G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [4]:
print(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(78464, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=512, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=512, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (n

## 4단계 - 잘 실행되는지 확인

In [5]:
device = "cuda:0"

messages = [
    {"role": "user", "content": "은행의 기준 금리에 대해서 설명해줘"}
]


encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")

model_inputs = encodeds.to(device)


generated_ids = model.generate(model_inputs, max_new_tokens=1000, do_sample=True)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])


No chat template is defined for this tokenizer - using the default template for the LlamaTokenizerFast class. If the default is not appropriate for your model, please set `tokenizer.chat_template` to an appropriate template. See https://huggingface.co/docs/transformers/main/chat_templating for more information.



<|startoftext|> [INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>>

은행의 기준 금리에 대해서 설명해줘 [/INST]

기본 금리로 가장 대중적이며 기본적인 이자율이 포함됩니다. 이자 금리에는 소비자 금융 상품, 신용 카드가 포함되는데, 신용 관리 서비스를 제공하는 신용 카드에는 이자율이 추가되거나 또는 할인되거나 또는 면제됩니다.<|endoftext|>


## 5단계- RAG 시스템 결합하기

In [6]:
# pip install시 utf-8, ansi 관련 오류날 경우 필요한 코드
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [7]:
!pip -q install langchain pypdf chromadb sentence-transformers faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.1/803.1 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 508.6/508.6 kB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 12.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 55.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 205.7/205.7 kB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 65.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.3/60.3 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 47.3 MB/s et

In [8]:
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from transformers import pipeline
from langchain.chains import LLMChain

text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.2,
    return_full_text=True,
    max_new_tokens=300,
)

prompt_template = """
### [INST]
Instruction: Answer the question based on your knowledge.
Here is context to help:

{context}

### QUESTION:
{question}

[/INST]
 """

koplatyi_llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

# Create prompt from prompt template
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

# Create llm chain
llm_chain = LLMChain(llm=koplatyi_llm, prompt=prompt)

In [9]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFLoader
from langchain.schema.runnable import RunnablePassthrough

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [21]:
파일명 = "고용노동부_보도자료_샘플.pdf"
loader = PyPDFLoader(f"/content/drive/MyDrive/data/{파일명}")
pages = loader.load_and_split()
pages

[Document(page_content='고용노동부보도참고자료\n보도시점 2023. 12. 29.(금) 배포 즉시\n30인 미만 사업장 주 52시간제 시행 \n계도기간 연장\n  고용노동부 (장관 이정식 )는 8시간 추가근로제 일몰에 따라 30인 미만 \n사업장에 부여한 계도기간 을 1년 연장(’24.1.1.~’24.12.31.) 한다. \n  고용노동부는 주 52시간제가 현장에 정착되어가고 있으나 , 소규모 사업장\n에서는  상시적인 인력난과 고금리 ·고물가 등 경제상황 으로 여전히  어려움 을 \n겪고 있다는 현장의 의견(중소기업 사업주 간담회 , 관계부처 회의 등)을 반영하여 \n계도기간을 연장한 것으로 , 이는 한시적 조치라고 밝혔다 . \n  계도기간 중 30인 미만 사업장은 장시간 관련 정기 근로감독 대상에서 \n제외된다. 또한 그 외 근로감독 또는 진정 등의 처리 과정에서 근로시간 \n한도 위반이 확인되더라도 필요시 추가적으로 3~6개월의 시정기회 를 제공\n한다. 계도기간 부여와 함께 장시간 노동 방지 및 근로자 건강권 보호를 \n위해 자가진단표를 배포하는 한편, 근로자건강센터 안내·홍보도 병행한다. \n   * 다만, 특별감독 , 고소‧고발 사건의 경우 계도기간 부여와 관계없이 즉시 사법처리\n  한편, 고용노동부는 사회적 대화가 복원된만큼 노사정 대화를 통해 합리\n적인 대안을 마련하고, 근로시간 제도 개편을 조속히 추진하여 조기에 계도\n기간을  종료할 수 있도록 할 계획이다 . \n담당 부서 노동개혁정책관 책임자 과  장 이지영(044-202-7994)\n임금근로시간정책과 담당자 사무관 이혜민(044-202-7541)', metadata={'source': '/content/drive/MyDrive/data/고용노동부_보도자료_샘플.pdf', 'page': 0})]

In [22]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
texts = text_splitter.split_documents(pages)

from langchain.embeddings import HuggingFaceEmbeddings

model_name = "jhgan/ko-sbert-nli"
encode_kwargs = {'normalize_embeddings': True}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    encode_kwargs=encode_kwargs
)

db = FAISS.from_documents(texts, hf)
retriever = db.as_retriever(
                            search_type="similarity",
                            search_kwargs={'k': 3}
                        )

In [23]:
rag_chain = (
 {"context": retriever, "question": RunnablePassthrough()}
    | llm_chain
)

In [24]:
import warnings
warnings.filterwarnings('ignore')

In [28]:
result = rag_chain.invoke("52시간제 시행 계도기간 연장 사유는 무엇인가?")

for i in result['context']:
    print(f"주어진 근거: {i.page_content} / 출처: {i.metadata['source']} - {i.metadata['page']} \n\n")

print(f"\n답변: {result['text']}")

주어진 근거: 고용노동부보도참고자료
보도시점 2023. 12. 29.(금) 배포 즉시
30인 미만 사업장 주 52시간제 시행 
계도기간 연장
  고용노동부 (장관 이정식 )는 8시간 추가근로제 일몰에 따라 30인 미만 
사업장에 부여한 계도기간 을 1년 연장(’24.1.1.~’24.12.31.) 한다. 
  고용노동부는 주 52시간제가 현장에 정착되어가고 있으나 , 소규모 사업장
에서는  상시적인 인력난과 고금리 ·고물가 등 경제상황 으로 여전히  어려움 을 
겪고 있다는 현장의 의견(중소기업 사업주 간담회 , 관계부처 회의 등)을 반영하여 
계도기간을 연장한 것으로 , 이는 한시적 조치라고 밝혔다 . 
  계도기간 중 30인 미만 사업장은 장시간 관련 정기 근로감독 대상에서 
제외된다. 또한 그 외 근로감독 또는 진정 등의 처리 과정에서 근로시간 
한도 위반이 확인되더라도 필요시 추가적으로 3~6개월의 시정기회 를 제공
한다. 계도기간 부여와 함께 장시간 노동 방지 및 근로자 건강권 보호를 / 출처: /content/drive/MyDrive/data/고용노동부_보도자료_샘플.pdf - 0 


주어진 근거: 한다. 계도기간 부여와 함께 장시간 노동 방지 및 근로자 건강권 보호를 
위해 자가진단표를 배포하는 한편, 근로자건강센터 안내·홍보도 병행한다. 
   * 다만, 특별감독 , 고소‧고발 사건의 경우 계도기간 부여와 관계없이 즉시 사법처리
  한편, 고용노동부는 사회적 대화가 복원된만큼 노사정 대화를 통해 합리
적인 대안을 마련하고, 근로시간 제도 개편을 조속히 추진하여 조기에 계도
기간을  종료할 수 있도록 할 계획이다 . 
담당 부서 노동개혁정책관 책임자 과  장 이지영(044-202-7994)
임금근로시간정책과 담당자 사무관 이혜민(044-202-7541) / 출처: /content/drive/MyDrive/data/고용노동부_보도자료_샘플.pdf - 0 



답변: 30인 미만 사업장에 부여한 계도기간을 1년 연장(’24.1.1.~’24.12.31

In [29]:
result = rag_chain.invoke("중요한 내용을 3문장 이하로 요약해주세요.")

for i in result['context']:
    print(f"주어진 근거: {i.page_content} / 출처: {i.metadata['source']} - {i.metadata['page']} \n\n")

print(f"\n답변: {result['text']}")

주어진 근거: 고용노동부보도참고자료
보도시점 2023. 12. 29.(금) 배포 즉시
30인 미만 사업장 주 52시간제 시행 
계도기간 연장
  고용노동부 (장관 이정식 )는 8시간 추가근로제 일몰에 따라 30인 미만 
사업장에 부여한 계도기간 을 1년 연장(’24.1.1.~’24.12.31.) 한다. 
  고용노동부는 주 52시간제가 현장에 정착되어가고 있으나 , 소규모 사업장
에서는  상시적인 인력난과 고금리 ·고물가 등 경제상황 으로 여전히  어려움 을 
겪고 있다는 현장의 의견(중소기업 사업주 간담회 , 관계부처 회의 등)을 반영하여 
계도기간을 연장한 것으로 , 이는 한시적 조치라고 밝혔다 . 
  계도기간 중 30인 미만 사업장은 장시간 관련 정기 근로감독 대상에서 
제외된다. 또한 그 외 근로감독 또는 진정 등의 처리 과정에서 근로시간 
한도 위반이 확인되더라도 필요시 추가적으로 3~6개월의 시정기회 를 제공
한다. 계도기간 부여와 함께 장시간 노동 방지 및 근로자 건강권 보호를 / 출처: /content/drive/MyDrive/data/고용노동부_보도자료_샘플.pdf - 0 


주어진 근거: 한다. 계도기간 부여와 함께 장시간 노동 방지 및 근로자 건강권 보호를 
위해 자가진단표를 배포하는 한편, 근로자건강센터 안내·홍보도 병행한다. 
   * 다만, 특별감독 , 고소‧고발 사건의 경우 계도기간 부여와 관계없이 즉시 사법처리
  한편, 고용노동부는 사회적 대화가 복원된만큼 노사정 대화를 통해 합리
적인 대안을 마련하고, 근로시간 제도 개편을 조속히 추진하여 조기에 계도
기간을  종료할 수 있도록 할 계획이다 . 
담당 부서 노동개혁정책관 책임자 과  장 이지영(044-202-7994)
임금근로시간정책과 담당자 사무관 이혜민(044-202-7541) / 출처: /content/drive/MyDrive/data/고용노동부_보도자료_샘플.pdf - 0 



답변: 30인 미만 사업장에 대한 주 52시간제 계도기간이 1년 연장됩니다. 계도기간 중 

In [30]:
result = rag_chain.invoke("향후 예상되는 내용은 무엇인가?")

for i in result['context']:
    print(f"주어진 근거: {i.page_content} / 출처: {i.metadata['source']} - {i.metadata['page']} \n\n")

print(f"\n답변: {result['text']}")

주어진 근거: 한다. 계도기간 부여와 함께 장시간 노동 방지 및 근로자 건강권 보호를 
위해 자가진단표를 배포하는 한편, 근로자건강센터 안내·홍보도 병행한다. 
   * 다만, 특별감독 , 고소‧고발 사건의 경우 계도기간 부여와 관계없이 즉시 사법처리
  한편, 고용노동부는 사회적 대화가 복원된만큼 노사정 대화를 통해 합리
적인 대안을 마련하고, 근로시간 제도 개편을 조속히 추진하여 조기에 계도
기간을  종료할 수 있도록 할 계획이다 . 
담당 부서 노동개혁정책관 책임자 과  장 이지영(044-202-7994)
임금근로시간정책과 담당자 사무관 이혜민(044-202-7541) / 출처: /content/drive/MyDrive/data/고용노동부_보도자료_샘플.pdf - 0 


주어진 근거: 고용노동부보도참고자료
보도시점 2023. 12. 29.(금) 배포 즉시
30인 미만 사업장 주 52시간제 시행 
계도기간 연장
  고용노동부 (장관 이정식 )는 8시간 추가근로제 일몰에 따라 30인 미만 
사업장에 부여한 계도기간 을 1년 연장(’24.1.1.~’24.12.31.) 한다. 
  고용노동부는 주 52시간제가 현장에 정착되어가고 있으나 , 소규모 사업장
에서는  상시적인 인력난과 고금리 ·고물가 등 경제상황 으로 여전히  어려움 을 
겪고 있다는 현장의 의견(중소기업 사업주 간담회 , 관계부처 회의 등)을 반영하여 
계도기간을 연장한 것으로 , 이는 한시적 조치라고 밝혔다 . 
  계도기간 중 30인 미만 사업장은 장시간 관련 정기 근로감독 대상에서 
제외된다. 또한 그 외 근로감독 또는 진정 등의 처리 과정에서 근로시간 
한도 위반이 확인되더라도 필요시 추가적으로 3~6개월의 시정기회 를 제공
한다. 계도기간 부여와 함께 장시간 노동 방지 및 근로자 건강권 보호를 / 출처: /content/drive/MyDrive/data/고용노동부_보도자료_샘플.pdf - 0 



답변: 30인 미만 사업장에 대한 계도기간이 연장되었으며, 계도기간 중 30인 미만 사업장